In [18]:
import io
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pybedtools
from pybedtools import BedTool
# import statsmodels.api as sm
import pylab as py
import math
from scipy import stats
from scipy.stats.stats import pearsonr
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from matplotlib.ticker import MaxNLocator, FormatStrFormatter
from pathlib import Path
import time
from datetime import datetime
import pickle
from statsmodels.stats.multitest import fdrcorrection
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y-%H-%M-%S")

In [19]:
#Simon's code: extracting the current experiment of saturation data out of an input file depicting all experiments

#############################################################################
# functions
#############################################################################

def PREPARE_curr_saturation_df(saturation_file):
    curr_saturation_df = pd.read_csv(saturation_file, sep='\t')
    # remove lines where value is null
    curr_saturation_df = curr_saturation_df[pd.notnull(curr_saturation_df['Value'])].copy()
    # remove lines where ALT is '-' (deletions)
    curr_saturation_df = curr_saturation_df[curr_saturation_df['Alt']!='-'].copy()
    # map chromosome to int
    if curr_saturation_df.dtypes['Chromosome'] == 'int64' or curr_saturation_df.dtypes['Chromosome'] == 'float64':
        curr_saturation_df['Chromosome'] = curr_saturation_df['Chromosome'].astype(int)
    curr_saturation_df['Chromosome'] = 'chr' + curr_saturation_df['Chromosome'].map(str)
    # give percentiles to all values
    curr_saturation_df['percentile'] = curr_saturation_df['Value'].rank(pct=True)
    return curr_saturation_df


def PREPARE_curr_saturation_df(saturation_file):
    curr_saturation_df = pd.read_csv(saturation_file, sep='\t')
    curr_saturation_df = curr_saturation_df[pd.notnull(curr_saturation_df['Value'])].copy()
    curr_saturation_df = curr_saturation_df[curr_saturation_df['Alt']!='-'].copy()
    if curr_saturation_df.dtypes['Chromosome'] == 'int64' or curr_saturation_df.dtypes['Chromosome'] == 'float64':
        curr_saturation_df['Chromosome'] = curr_saturation_df['Chromosome'].astype(int)
    curr_saturation_df['Chromosome'] = 'chr' + curr_saturation_df['Chromosome'].map(str)
    curr_saturation_df['percentile'] = curr_saturation_df['Value'].rank(pct=True)
    return curr_saturation_df


In [20]:
#Simon's code

# This script analyzes saturation mutagenesis MPRA data.
# Inputs:
# 1. Variant catalogs - a list of variants from human - ape or MH/AH-derived catalogs.
# 2. A list of regions that represent each saturation experiment
# 3. Saturation valuesz
# It performs comparisons on the effect change of saturation data, focusing on the catalog variants.
# All coordinates here are bed-style - double check

project_dir = "/home/labs/davidgo/omerro/SaturationMutagenesis/main/Analysis/"
saturation_data_dir = project_dir + 'saturation_files_Yoav/'
output_dir = project_dir + 'output/'
# p_value_threshold = 0.05



# catalog_config_df is used to define which variant catalogs are used.
catalog_config_file = project_dir + 'catalog_config_Omer_1.txt'
catalog_config_df = pd.read_csv(catalog_config_file, sep='\t')
catalog_config_df = catalog_config_df.drop(catalog_config_df[catalog_config_df.use == 0].index)
# catalog_config_df = catalog_config_df.iloc[0:4].copy() # for now ignore MH/AH -derived catalogs

In [21]:
catalog_config_df

,catalog,path,use
0,human,/home/labs/davidgo/Collaboration/Tomas_Apes_Ca...,1
1,chimp,/home/labs/davidgo/Collaboration/Tomas_Apes_Ca...,1
2,gorilla,/home/labs/davidgo/Collaboration/Tomas_Apes_Ca...,1
3,pongo,/home/labs/davidgo/Collaboration/Tomas_Apes_Ca...,1
4,human_chimp,/home/labs/davidgo/Collaboration/Tomas_Apes_Ca...,1


In [22]:
#Simon's code - violin plots

# fill colors for the violin plots
catalog_config_df['edgecolor'] = 'grey'
catalog_config_df.loc[catalog_config_df['catalog'] == 'human', 'edgecolor'] = 'red'
catalog_config_df.loc[catalog_config_df['catalog'] == 'chimp', 'edgecolor'] = 'brown'
catalog_config_df.loc[catalog_config_df['catalog'] == 'gorilla', 'edgecolor'] = 'black'
catalog_config_df.loc[catalog_config_df['catalog'] == 'pongo', 'edgecolor'] = 'orange'
catalog_config_df.loc[catalog_config_df['catalog'] == 'human_chimp', 'edgecolor'] = 'purple'
# catalog_config_df['edgecolor'] = ['red','brown','black','orange']
catalog_config_df['total_variants_count'] = 0
catalog_config_df['overlapping_variants_count'] = 0

uc_df = pd.read_csv(r'/home/labs/davidgo/omerro/SaturationMutagenesis/main/Library_assembly/UC/UC_table.csv', header=[0, 1], index_col=0)
uc_df_lifted = pd.read_csv(r'/home/labs/davidgo/omerro/SaturationMutagenesis/main/Library_assembly/UC/lifted_uc.bed'
                           , sep="\t", names=['chr', 'start', 'end', 'prev', 'num'], dtype={'start': np.int32,'end': np.int32})
uc_df_filter=uc_df[[('ultra conserved element', 'name'),('ultra conserved element', 'type')]].reset_index()
uc_df_final = pd.concat([uc_df_lifted[['chr', 'start', 'end']], uc_df_filter.iloc[:,1:3]].copy(), axis=1)
uc_df_final.columns = ['chr', 'start', 'end', 'UC_ID','type']
uc_df_final=uc_df_final[uc_df_final['type']=='n']
# regions_df - a list of regions, one region for each saturation experiment
regions_df = uc_df_final
regions_bed = pybedtools.BedTool.from_dataframe(regions_df)
regions_df['length'] = regions_df['end'] - regions_df['start']

In [45]:
uc_df_final[uc_df_final['UC_ID']=="uc.88"]

,chr,start,end,UC_ID,type,length
85,chr2,162095041,162095353,uc.88,n,312


In [ ]:
#Simon's code - mapping the catalogs to the saturation data

intersect_dict = {}
# a loop on each catalog - species group. creates a dictionary of dataframes.
# each df in the dictionary is an intersect between a specific variant catalog and all the saturation regions
for index, catalog in catalog_config_df.iterrows():
    print(index)
    catalog_file = catalog['path']
    catalog_df = pd.read_csv(catalog_file, sep='\t', header=None)
    catalog_bed = pybedtools.BedTool.from_dataframe(catalog_df)
    curr_intersect = catalog_bed.intersect(regions_bed, wb=True)
    intersect_df = curr_intersect.to_dataframe()
    if len(intersect_df) > 0:
        intersect_df['pos'] = intersect_df['start'] + 1
        intersect_df['REF'] = intersect_df['name'].str.get(0)
        intersect_df['ALT'] = intersect_df['name'].str.get(-1)
    catalog_config_df.at[index, 'total_variants_count'] = len(catalog_df)
    catalog_config_df.at[index, 'overlapping_variants_count'] = len(intersect_df)
    intersect_dict[catalog['catalog']] = intersect_df

    




In [17]:
df_list=[]
source=[]
for key,val in intersect_dict.items():
    source.extend([key for i in range(len(val))])
    df_list.append(val)

NameError: name 'intersect_dict' is not defined

In [16]:
df=pd.concat(df_list)
df['source']=source

SyntaxError: invalid syntax (3186785960.py, line 1)

In [ ]:
df.to_csv(r'/home/labs/davidgo/omerro/SaturationMutagenesis/main/Library_assembly/UC/UC_variants.csv')

In [26]:
df=pd.read_csv(r'/home/labs/davidgo/omerro/SaturationMutagenesis/main/Library_assembly/UC/UC_variants.csv',index_col=0)

In [27]:
df

,chrom,start,end,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,pos,REF,ALT,source
0,chr2,144715351,144715352,T->C,.,chr2,144714971,144715378,uc.72,n,144715352,T,C,human
0,chr1,215889140,215889141,G->A,.,chr1,215889031,215889297,uc.42,n,215889141,G,A,chimp
1,chr10,102419478,102419479,A->G,.,chr10,102419219,102419583,uc.297,n,102419479,A,G,chimp
2,chr10,124852762,124852763,G->A,0.000350229,chr10,124852614,124852849,uc.315,n,124852763,G,A,chimp
3,chr12,54090866,54090867,G->A,.,chr12,54090831,54091090,uc.340,n,54090867,G,A,chimp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,chr9,23496834,23496835,A->C,.,chr9,23496724,23497003,uc.254,n,23496835,A,C,pongo
59,chr9,37215204,37215205,T->A,.,chr9,37215203,37215467,uc.257,n,37215205,T,A,pongo
60,chr9,128432732,128432733,A->G,.,chr9,128432587,128432800,uc.272,n,128432733,A,G,pongo
61,chrX,24916919,24916920,C->T,.,chrX,24916157,24916927,uc.464,n,24916920,C,T,pongo


In [38]:
df[df['source']=="human"]

,chrom,start,end,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,pos,REF,ALT,source
0,chr2,144715351,144715352,T->C,.,chr2,144714971,144715378,uc.72,n,144715352,T,C,human


In [40]:
df[df['itemRgb']=="uc.88"]

,chrom,start,end,name,score,strand,thickStart,thickEnd,itemRgb,blockCount,pos,REF,ALT,source
